<a href="https://colab.research.google.com/github/BartoszJanJerzy/M-L-Bootcamp-cz.1/blob/master/02_braki_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sklearn

##Tworzenie danych

In [4]:
data = {
    'size': ['XL','L','M', np.nan, 'M','M'],
    'color':['red','green','blue','green','red', 'green'],
    'gender':['female','male', np.nan,'female','female','male'],
    'price':[199.0,89.0, np.nan, 129.0,79.0, 89.0],
    'weight':[500,450,300, np.nan,410, np.nan,],
    'bought':['yes','no','yes','no','yes', 'no']
}

df_raw = pd.DataFrame(data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,NaN,green,female,129.0,NaN,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,NaN,no


In [18]:
df = df_raw.copy()

##Sprawdzenie braków

In [7]:
df.isnull()
# false - wartość
# true - brak danych

,size,color,gender,price,weight,bought
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,True,True,False,False
3,True,False,False,False,True,False
4,False,False,False,False,False,False
5,False,False,False,False,True,False


In [8]:
# a ile ich w poszczególnych kolumnach jest?
df.isnull().sum()

size      1
color     0
gender    1
price     1
weight    2
bought    0
dtype: int64

In [9]:
# a ile jest ich w całym df?
df.isnull().sum().sum()

5

In [10]:
# a procentowwo w kolumnach?
df.isnull().sum() / len(df)

size      0.166667
color     0.000000
gender    0.166667
price     0.166667
weight    0.333333
bought    0.000000
dtype: float64

In [19]:
# mozna też wyszukać konkrente wiersze z brakami
filter = pd.isnull(df['weight'])
df[filter]
# znak ~filter to negacja

,size,color,gender,price,weight,bought
3,NaN,green,female,129.0,NaN,no
5,M,green,male,89.0,NaN,no


In [20]:
# a metoda, która pokazuje, czy komórka ma wartość, czy brak? 
# odwrotnie jak isnull()
df.notnull()

,size,color,gender,price,weight,bought
0,True,True,True,True,True,True
1,True,True,True,True,True,True
2,True,True,False,False,True,True
3,False,True,True,True,False,True
4,True,True,True,True,True,True
5,True,True,True,True,False,True


##Uzupełnienie braków jakaś statystyką z biblioteki SKLEARN

In [12]:
from sklearn.impute import SimpleImputer

# strategy: 'mean','medain','most_frequent','constant'
# jeżeli wybierze się strategię 'constant', to trzeba jeszcze utalić parametr fill_value=
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df[['price','weight']] = imputer.fit_transform(df[['price','weight']])

In [16]:
imputer.statistics_

array([117., 415.])

## Uzupełnianie braków z bilbioteki PANDAS

In [25]:
# przykładowo: 'np.nan' zastąpić kategorią 'brak'
df['size'].fillna(value='brak', inplace=True)
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,NaN,NaN,300.0,yes
3,brak,green,female,129.0,NaN,no
4,M,red,female,79.0,410.0,yes
5,M,green,male,89.0,NaN,no


##Braki danych w szeregach czasowych

Twrzenie danych

In [38]:
data = {'price': [108,109,110,110,109, np.nan, np.nan, 112,111,111]}
data_range = pd.date_range(start='01-01-2020 09:00', end='01-01-2020 18:00', periods=10)

df = pd.DataFrame(data=data, index=data_range)
df

,price
2020-01-01 09:00:00,108.0
2020-01-01 10:00:00,109.0
2020-01-01 11:00:00,110.0
2020-01-01 12:00:00,110.0
2020-01-01 13:00:00,109.0
2020-01-01 14:00:00,NaN
2020-01-01 15:00:00,NaN
2020-01-01 16:00:00,112.0
2020-01-01 17:00:00,111.0
2020-01-01 18:00:00,111.0


Wizualizacja danych

In [39]:
import plotly.graph_objects as go

In [40]:
df.index

DatetimeIndex(['2020-01-01 09:00:00', '2020-01-01 10:00:00',
               '2020-01-01 11:00:00', '2020-01-01 12:00:00',
               '2020-01-01 13:00:00', '2020-01-01 14:00:00',
               '2020-01-01 15:00:00', '2020-01-01 16:00:00',
               '2020-01-01 17:00:00', '2020-01-01 18:00:00'],
              dtype='datetime64[ns]', freq=None)

In [77]:
def draw_graph(x_values,y_values, title):
  fig = go.Figure(data = go.Scatter(x=x_values,
                                    y=y_values,
                                    mode='lines + markers'),
                  layout = go.Layout(width=1000,
                                    height=400,
                                    title=title))

  # fig.update_yaxes(rangemode='tozero')

  fig.show()

draw_graph(df.index,df.price, 'Szeregi czasowe z brakiem danych')

In [78]:
# a gdyby usunąc rekordy z brakiem danych?
df_temp = df.copy()
df_temp.dropna(inplace=True)

draw_graph(df_temp.index,df_temp.price, 'Szeregi czasowe z usuniętymi brakami danych')

In [79]:
# a gdyby wypełnić braki jakąś stałą wartością?
df_temp = df.copy()
df_temp['price'].fillna(0, inplace=True)

draw_graph(df_temp.index,df_temp.price, 'Szeregi czasowe uzupełnionymi brakami wartością 0')

In [80]:
# a gdyby wypełnić braki średnią?

df_temp = df.copy()

# sklearn
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df_temp[['price']] = imputer.fit_transform(df_temp[['price']])

# pandas
df_temp = df.copy()
df_temp['price'].fillna(df_temp['price'].mean(), inplace=True)


draw_graph(df_temp.index,df_temp.price, 'Szeregi czasowe uzupełnionymi brakami średnią')

In [81]:
# a metoda interpolacji
df_temp = df.copy()
df_temp['price'] = df_temp['price'].interpolate()

draw_graph(df_temp.index,df_temp.price, 'Szeregi czasowe z brakami uzupełnionymi metodą interpolacji')

In [83]:
# metoda forward_fill
df_temp = df.copy()
df_temp['price'].fillna(method='ffill', inplace=True)

draw_graph(df_temp.index,df_temp.price, 'Szeregi czasowe z brakami uzupełnionymi metodą forward_fill')

In [85]:
# metoda backward_fill
df_temp = df.copy()
df_temp['price'].fillna(method='bfill', inplace=True)

draw_graph(df_temp.index,df_temp.price, 'Szeregi czasowe z brakami uzupełnionymi metodą backward_fill')